<span style='color:gray'> <span style="font-size:25px;"> **Development of "Machine Learning Models"  (Workflow)**
    
In this Notebook, the machine learning model will be created and then the data from well-logs DLIS file [after preprocessing, sorting and finalizing the data] is loaded as input for Machine Learning model (ML); 
* Random Forest Regressor
* Gradient Boosting Regressor
    
    
For the prediction of petrophysical properties, such as porosity, permeability and water saturation, these two Regressor models **Random Forest Regressor** and **Gradient Boosting Regressor** are suitable.

They are Ensemble Based Tree Methods; they are based on the generation of Decision Trees.

We use Regression Models since we want to predict a continuous variable.

**Advantages** of the 2 regression models, since they are based on Decision Trees:

* They do not need the normalization or scaling of the original dataset;
* They are not sensitive to outliers, thus, outliers detection and removal are not required.

**==================================================================================================================**
    
In well-log machine learning models, the choice between regression and classification (Supervised ML) depends on the nature of the problem you are trying to solve and the type of data you have. Let's break down the reasons why regression is often preferred over classification in this context:

**Continuous Output**: Well-log data often involves continuous measurements such as porosity, permeability, resistivity, and other geological properties. Regression is well-suited for predicting and modeling continuous numerical values. Classification, on the other hand, is typically used when the output is categorical or discrete, like classifying lithology or rock types.

**Data Distribution**: Well-log data tends to have a wide range of continuous values. Using classification would require discretizing this data into bins or classes, which can lead to loss of information and potentially introduce biases. Regression models can capture the nuances and variations present in the continuous data more effectively.

**Evaluation Metrics**: Regression models are evaluated using metrics such as mean squared error (MSE), root mean squared error (RMSE), or mean absolute error (MAE). These metrics are well-suited for measuring the accuracy of predictions involving continuous values. Classification models, on the other hand, use metrics like accuracy, precision, recall, and F1-score, which are designed for categorical predictions.

**Feature Importance**: Well-log data analysis often involves understanding the relationships between different geological features and the target property. Regression models can provide insights into the quantitative impact of each feature on the predicted values, aiding in geological interpretation.

<span style='color:gray'> <span style="font-size:20px;"> 
**Importing Libraries, Regressors, and Required Dependencies**

In [1]:
%pip install --quiet --upgrade scikit-learn==1.2.2
%pip install --quiet qbstyles


# Importing the dependencies
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

from qbstyles import mpl_style
mpl_style(dark=False)  # Set light matplotlib style

import matplotlib.patches as mpatches  # To create a legend with a color box
import pickle

# Importing the models 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neural_network import MLPRegressor
                                         
from sklearn.model_selection import RandomizedSearchCV

# train_test_split is a function 
# cross_val_score and KFold are functions

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold 

# Regression metrics
from sklearn.metrics import r2_score, mean_squared_error, mean_squared_error, mean_absolute_error

# The package "Matplotlib Inline Back-end" provides support for Matplotlib to display figures directly inline
# "svg" stands for "scalable vector graphic". The plot can be scaled without compromising its quality
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg') 

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


<span style='color:gray'> <span style="font-size:20px;"> 
**Loading datafile that has been extracted from DLIS or LAS file after (Sorting, Cleaning, preprocessing, choosing the logs based on need logically ...)**

Load the **csv** well log data to Pandas DataFrame 

In [2]:
file_path = '/Users/amirhosseinakhondzadeh/CODE_WELLLOGS/Petrobras Well-log Analysis/Processed Data (out put of preprocessing == Input of ML)/df0_ML.csv'
df = pd.read_csv(file_path)
df

,DEPTH,GR,RHGX_HILT,NPHI,AT10,AT20,AT30,AT60,AT90,PEFZ
0,3241.2432,43.603180,2.831039,0.011302,51.794643,61.238102,52.368523,47.517567,38.26941,3.912346
1,3241.3955,43.603180,2.831039,0.011302,51.794643,61.238102,52.368523,47.517567,38.26941,3.912346
2,3241.5480,31.196218,2.831039,0.011302,51.794643,61.238102,52.368523,47.517567,38.26941,3.912346
3,3241.7004,22.927324,2.831039,0.011413,51.794643,61.238102,52.368523,47.517567,38.26941,3.912346
4,3241.8528,25.734980,2.832985,0.011976,51.794643,61.238102,52.368523,47.517567,38.26941,3.912346
...,...,...,...,...,...,...,...,...,...,...
1141,3415.1316,219.444870,2.834162,0.091268,6.251309,1950.000000,101.902054,419.289830,207.16121,3.492837
1142,3415.2840,219.444870,2.833239,0.091268,6.248991,1950.000000,101.493515,412.861100,209.38712,3.481267
1143,3415.4365,219.444870,2.832363,0.091268,6.247348,1950.000000,101.377750,416.689060,214.43398,3.470307
1144,3415.5889,219.444870,2.831719,0.091268,6.247392,1950.000000,101.574510,427.813420,221.12366,3.467056


<span style='color:gray'> <span style="font-size:20px;"> 
**Defining Predictor (X) and what will be predicted**

In [3]:
predictors = ["GR","NPHI"] 
X = df[predictors]
y = df["PEFZ"]

<span style='color:gray'> <span style="font-size:20px;"> 
    
**Training & Test Well Log Datasets**

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

* **Data Splitting Function**: We utilize the "train_test_split" function to perform the splitting of our data.


* **Variables to Split**: The data we're working with consists of two main variables, denoted as X and y. These are the entities that we want to partition.


* **Training Set Size**: Instead of specifying an exact training set size, we have the option to leave it as "None." In this case, the function will automatically determine the training size based on the complement of the test size. The test size is set at 20%, meaning 80% will be allocated to the training set.


* **Test Set Size**: We assign a test size of 20%, indicating that one-fifth of the entire dataset will be allocated for testing the model's performance.


* **Shuffling Data**: The default behavior is to shuffle the data prior to splitting. This randomization helps in creating a balanced distribution between the training and test sets.


* **Reproducibility with Random Seed**: For the sake of reproducibility across multiple runs of the function, we introduce an integer value known as the "random state." Here, we've chosen the value 42. It's essential to set this only when shuffling is enabled.


*In essence, we're utilizing the "train_test_split" function to divide our data into training and test portions. We provide our data variables X and y, and the function handles the allocation. The training size is determined as the complementary value to the test size, which is set at 20%. Shuffling the data ensures randomness, and to achieve consistent outcomes in different runs, we use a random state value of 42, applying it only when shuffling is activated.*

In [5]:
print(X.shape, y.shape, X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1146, 2) (1146,) (916, 2) (230, 2) (916,) (230,)


<span style='color:gray'> <span style="font-size:20px;"> 
**Cross-Validation: Enhancing Model Evaluation**

<span style='color:black'> <span style='font-size:14px;'> **Cross-validation is a technique used to evaluate the performance of a machine learning model. It works by dividing the training dataset into k subsets, called folds. The model is then trained on k-1 folds of the training dataset and evaluated on the remaining fold. This procedure is repeated k times, with each fold being used as the validation set once. The average accuracy of the model across all k folds is then used as an estimate of the model's overall performance.**</span></span>

* It is proposed to use k-fold cross-validation to evaluate the performance of two models on the training dataset. This will help them to choose the model that is most likely to generalize well to unseen data.

* The k-fold cross-validation step can be skipped, since it will be carried out again during the optimization process. However, it is still a good idea to perform k-fold cross-validation on the training dataset before starting the optimization process, as this will help to ensure that the optimization process is not converging to a local optimum.

Here are some additional details about k-fold cross-validation:

- The value of k is typically chosen to be between 5 and 10.
- The folds should be created randomly, to avoid any bias in the results.
- The model should be trained and evaluated on the same set of features for each fold.
- The accuracy of the model is typically measured using a metric such as the coefficient of determination.


K-fold cross-validation is a powerful technique for evaluating the performance of machine learning models. It is more reliable than simply training and evaluating the model on a single holdout dataset, as it helps to mitigate the effects of overfitting.

<span style='color:gray'> <span style="font-size:20px;"> 
**Creating Models + Cross-Validation [evaluation the performance of a machine learning model]**

Creating Models such as **Random Forest** & **Gradient Boosting** Models

using **for loop** to iterate over different models and compare them together

In [6]:
rf_model = RandomForestRegressor(random_state=42)  # Random Forest Model 
gb_model = GradientBoostingRegressor(random_state=42)  # Gradient Boosting Model 

models = [rf_model, gb_model]

kf = KFold(n_splits=10, shuffle=True, random_state=42)  # Number of folds 

def compare_models_cv():  
    for model in models:
        r2_score = cross_val_score(model, X_train, y_train, cv=kf, scoring='r2')
        r2_score = np.round(r2_score,4)
        mean_r2 = sum(r2_score)/len(r2_score)
        mean_r2 = mean_r2*100
        mean_r2 = round(mean_r2,2)

        print('Coefficient of Determination for', model, '=', r2_score)
        print('Average % Coefficient of Determination for', model, '=', mean_r2)
        print('============================================')
        
compare_models_cv() 

Coefficient of Determination for RandomForestRegressor(random_state=42) = [0.6216 0.709  0.7017 0.4652 0.6325 0.5615 0.5508 0.5754 0.698  0.5291]
Average % Coefficient of Determination for RandomForestRegressor(random_state=42) = 60.45
Coefficient of Determination for GradientBoostingRegressor(random_state=42) = [0.6433 0.7415 0.6819 0.4158 0.5621 0.5719 0.5457 0.5043 0.72   0.5193]
Average % Coefficient of Determination for GradientBoostingRegressor(random_state=42) = 59.06


<span style='color:gray'> <span style="font-size:20px;"> 
**Hyperparameter Tuning (Randomized Search CV) - Optimization Problem**

We re-consider the training dataset and use the Randomized Search Cross Validation technique to determine **the optimal hyperparameter values** for <span style='color:blue'> <span style="font-size:15px;"> the Random Forest</span> </span>
and
<span style='color:blue'> <span style="font-size:15px;"> Gradient Boosting models </span> </span>.

*To start, we define a grid of hyperparameters that will be randomly sampled when calling the RandomizedSearchCV() function. The models are then cross-validated on these random combinations of hyperparameters.*

**The parameters of the RandomizedSearchCV() function are:**

* The model without any hyperparameters
* The grid of hyperparameters
* The number of combinations to be randomly sampled (n_iter=20)
* The number of k-folds into which the training dataset is split (cv=10)
* The technique returns the optimal combination of hyperparameters for the two models.

**Here is a more detailed explanation of each parameter:**

* **Model**: The model without any hyperparameters is the base model that we will use to start the search. In this case, we are using the Random Forest and Gradient Boosting models.
* **Grid of hyperparameters**: The grid of hyperparameters defines the range of values that we will randomly sample from. This allows us to explore a wider range of hyperparameter values than if we were to simply grid search over a fixed set of values.
* **Number of combinations to be randomly sampled (n_iter=20)**: The n_iter parameter specifies the number of random combinations of hyperparameters to be sampled. In this case, we are sampling 20 combinations.
* **Number of k-folds into which the training dataset is split (cv=10)**: The cv parameter specifies the number of k-folds to use for cross-validation. In this case, we are using 10 folds.
* The RandomizedSearchCV() function will randomly sample 20 combinations of hyperparameters from the grid and cross-validate each combination on 10 folds of the training dataset. The function will then return the combination of hyperparameters that resulted in the best cross-validation score.

This technique allows us to quickly and efficiently explore a wide range of hyperparameter values to find the optimal combination for our models.

<span style='color:red'> <span style="font-size:15px;"> 
**Random Forest Model**

We consider the following **hyperparameters**:

* n_estimators = number of trees in the forest; **(Number of trees to be used)**
* max_depth = the maximum depth of the tree; **(Maximum number of levels in tree)**
* criterion = the function that measures the quality of the split; **(Criterion to split on)**

<span style='color:blue'> <span style="font-size:15px;"> **then Grid Creation**:</span> </span>  grid of Hyperparameters 

In [7]:
rf_n_estimators = [100, 150, 200, 250, 300, 350, 400]
rf_max_depth = [5, 10, 15, 20, 25]
rf_criterion = ['squared_error']                         # "squared_error" is by default. It is optional

rf_grid = {'n_estimators': rf_n_estimators, 'max_depth': rf_max_depth, 'criterion': rf_criterion}

In the line below:

the RandomizedSearchCV class in scikit-learn is used **to tune the hyperparameters of a RandomForestRegressor model**.

* The first line of code creates a **RandomForestRegressor model** with a random state of 42. This ensures that the results of the model are reproducible.


* The second line of code creates a **RandomizedSearchCV object**. This object will be used to search for the best hyperparameters for the RandomForestRegressor model.


<span style='color:blue'> <span style="font-size:15px;">**rf_model**</span> </span>:
The model to be tuned. In this case, it is a RandomForestRegressor model.

<span style='color:blue'> <span style="font-size:15px;">**rf_grid**</span> </span>: 
A dictionary of hyperparameters to search. The keys of the dictionary are the hyperparameter names, and the values are the possible values to search for. In this case, the dictionary is searching for the best values of the n_estimators and max_depth hyperparameters.

<span style='color:blue'> <span style="font-size:15px;">**n_iter**</span> </span>:
The number of hyperparameter combinations to try. In this case, the RandomizedSearchCV object will try 20 different combinations of hyperparameters.

<span style='color:blue'> <span style="font-size:15px;">**cv**</span> </span>:
The number of folds to use for cross-validation. In this case, the RandomizedSearchCV object will use 10 folds for cross-validation.

<span style='color:blue'> <span style="font-size:15px;">**random_state**</span> </span>:
The random state to use for the search. In this case, the random state is set to 42.

In [8]:
rf_model = RandomForestRegressor(random_state=42)        # Shuffle=True by default

rf_random = RandomizedSearchCV(rf_model, rf_grid, n_iter=20, cv=10, random_state=42)

rf_random.fit(X_train, y_train)

RandomizedSearchCV(cv=10, estimator=RandomForestRegressor(random_state=42),
                   n_iter=20,
                   param_distributions={'criterion': ['squared_error'],
                                        'max_depth': [5, 10, 15, 20, 25],
                                        'n_estimators': [100, 150, 200, 250,
                                                         300, 350, 400]},
                   random_state=42)

In [9]:
rf_random.cv_results_

{'mean_fit_time': array([0.57844732, 0.57145364, 0.42021604, 0.16675055, 0.24206901,
        0.24825983, 0.56266639, 0.49961836, 0.21404765, 0.32124128,
        0.28432901, 0.58360012, 0.11554503, 0.34544241, 0.40203595,
        0.6572217 , 0.40663614, 0.42638006, 0.1754765 , 0.23194132]),
 'std_fit_time': array([0.00414218, 0.00605534, 0.01125167, 0.00152174, 0.0024636 ,
        0.00119596, 0.00292103, 0.00892965, 0.00149809, 0.00286689,
        0.0018157 , 0.01224747, 0.00067212, 0.00338455, 0.00237903,
        0.0033127 , 0.01072229, 0.00292729, 0.00415772, 0.00149106]),
 'mean_score_time': array([0.02336271, 0.02477155, 0.01756394, 0.00756934, 0.01041541,
        0.01059754, 0.02311478, 0.02199676, 0.0098233 , 0.01343198,
        0.01278563, 0.02397602, 0.00643823, 0.01690478, 0.016594  ,
        0.0264854 , 0.01967926, 0.01871829, 0.009073  , 0.01177993]),
 'std_score_time': array([0.0002336 , 0.00056496, 0.0012329 , 0.00039915, 0.00026764,
        0.00011665, 0.00021786, 0.001670

<span style='color:blue'> <span style="font-size:15px;"> **Print the best hyperparameters**:</span> </span>

Combination that gives **the highest accuracy (coefficient of determination) during the cross-validation**

In [10]:
best_parameters = rf_random.best_params_
print(best_parameters) 

{'n_estimators': 300, 'max_depth': 5, 'criterion': 'squared_error'}


Print the accuracy we get for the 
<span style='color:green'> <span style="font-size:15px;"> **best combination**:</span> </span>

In [11]:
highest_accuracy = rf_random.best_score_ 
highest_accuracy = round(highest_accuracy,4)
highest_accuracy = highest_accuracy*100
print('Coefficient of Determination for the Random Forest =', highest_accuracy)

Coefficient of Determination for the Random Forest = 60.33


<span style='color:green'> <span style="font-size:15px;"> *Load the results to Pandas DataFrame*:</span> </span>

In [12]:
results = pd.DataFrame(rf_random.cv_results_) 
results.head(20)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_max_depth,param_criterion,params,split0_test_score,split1_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.578447,0.004142,0.023363,0.000234,350,20,squared_error,"{'n_estimators': 350, 'max_depth': 20, 'criter...",0.241589,0.595431,...,0.753959,0.728023,0.645336,0.477538,0.642300,0.761411,0.573422,0.590287,0.150572,14
1,0.571454,0.006055,0.024772,0.000565,400,10,squared_error,"{'n_estimators': 400, 'max_depth': 10, 'criter...",0.261350,0.606643,...,0.766599,0.731700,0.655378,0.485465,0.631629,0.774841,0.575609,0.599355,0.147419,7
2,0.420216,0.011252,0.017564,0.001233,250,20,squared_error,"{'n_estimators': 250, 'max_depth': 20, 'criter...",0.248591,0.585951,...,0.755003,0.728726,0.641140,0.481904,0.649841,0.759321,0.573925,0.589823,0.149450,15
3,0.166751,0.001522,0.007569,0.000399,100,20,squared_error,"{'n_estimators': 100, 'max_depth': 20, 'criter...",0.248797,0.580827,...,0.750966,0.723731,0.645523,0.447419,0.658330,0.749416,0.582318,0.585643,0.151089,20
4,0.242069,0.002464,0.010415,0.000268,150,15,squared_error,"{'n_estimators': 150, 'max_depth': 15, 'criter...",0.248245,0.591811,...,0.747371,0.724771,0.645208,0.480516,0.643698,0.754856,0.587051,0.589203,0.148185,17
5,0.248260,0.001196,0.010598,0.000117,150,25,squared_error,"{'n_estimators': 150, 'max_depth': 25, 'criter...",0.245025,0.592762,...,0.748700,0.724742,0.645609,0.477871,0.645865,0.751769,0.584311,0.588292,0.149207,19
6,0.562666,0.002921,0.023115,0.000218,350,15,squared_error,"{'n_estimators': 350, 'max_depth': 15, 'criter...",0.244695,0.596217,...,0.754919,0.729951,0.645658,0.481330,0.642519,0.763111,0.575638,0.592051,0.149858,11
7,0.499618,0.008930,0.021997,0.001670,350,10,squared_error,"{'n_estimators': 350, 'max_depth': 10, 'criter...",0.265946,0.603373,...,0.765542,0.730504,0.655346,0.483713,0.633998,0.776508,0.578951,0.600007,0.146349,3
8,0.214048,0.001498,0.009823,0.000175,150,10,squared_error,"{'n_estimators': 150, 'max_depth': 10, 'criter...",0.276275,0.602515,...,0.756748,0.723289,0.656181,0.486800,0.636696,0.770596,0.587052,0.598024,0.143024,10
9,0.321241,0.002867,0.013432,0.000163,200,15,squared_error,"{'n_estimators': 200, 'max_depth': 15, 'criter...",0.251104,0.588787,...,0.749809,0.729792,0.637884,0.489440,0.642774,0.757348,0.583739,0.589715,0.147792,16


<span style='color:green'> <span style="font-size:15px;"> *Define the columns of interest*:</span> </span>

In [13]:
results_of_interest = results[['param_n_estimators','param_max_depth','param_criterion','params','mean_test_score']]
results_of_interest.head(20)

,param_n_estimators,param_max_depth,param_criterion,params,mean_test_score
0,350,20,squared_error,"{'n_estimators': 350, 'max_depth': 20, 'criter...",0.590287
1,400,10,squared_error,"{'n_estimators': 400, 'max_depth': 10, 'criter...",0.599355
2,250,20,squared_error,"{'n_estimators': 250, 'max_depth': 20, 'criter...",0.589823
3,100,20,squared_error,"{'n_estimators': 100, 'max_depth': 20, 'criter...",0.585643
4,150,15,squared_error,"{'n_estimators': 150, 'max_depth': 15, 'criter...",0.589203
5,150,25,squared_error,"{'n_estimators': 150, 'max_depth': 25, 'criter...",0.588292
6,350,15,squared_error,"{'n_estimators': 350, 'max_depth': 15, 'criter...",0.592051
7,350,10,squared_error,"{'n_estimators': 350, 'max_depth': 10, 'criter...",0.600007
8,150,10,squared_error,"{'n_estimators': 150, 'max_depth': 10, 'criter...",0.598024
9,200,15,squared_error,"{'n_estimators': 200, 'max_depth': 15, 'criter...",0.589715


======================================================================================================================

<span style='color:red'> <span style="font-size:15px;"> 
**Gradient Boosting Model**

We consider the following **hyperparameters**:

* n_estimators = number of trees in the forest; **(Number of trees to be used)**
* max_depth = the maximum depth of the tree; **(Maximum number of levels in tree)**
* criterion = the function that measures the quality of the split; **(Criterion to split on)**
* Learning rate 


<span style='color:blue'> <span style="font-size:15px;"> **then Grid Creation**:</span> </span>  grid of Hyperparameters 

GRADIENT BOOSTING Hyperparameters

In [14]:
gb_n_estimators = [100, 150, 200, 250, 300, 350, 400]
gb_max_depth = [5, 10, 15, 20, 25]
gb_rate = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
gb_criterion = ['squared_error']                       # It is optional 

gb_grid = {'n_estimators': gb_n_estimators,
           'max_depth': gb_max_depth,
           'learning_rate': gb_rate,
           'criterion': gb_criterion}

In the line below:

the RandomizedSearchCV class in scikit-learn is used **to tune the hyperparameters of a RandomForestRegressor model**.

* The first line of code creates a **RandomForestRegressor model** with a random state of 42. This ensures that the results of the model are reproducible.


* The second line of code creates a **RandomizedSearchCV object**. This object will be used to search for the best hyperparameters for the RandomForestRegressor model.

In [ ]:
gb_model = GradientBoostingRegressor(random_state=42) # Shuffle=True by default 

gb_random = RandomizedSearchCV(gb_model, gb_grid, n_iter=20, cv=10, random_state=42)

gb_random.fit(X_train, y_train)

In [ ]:
gb_random.cv_results_

<span style='color:blue'> <span style="font-size:15px;"> **Print the best hyperparameters**:</span> </span>

Combination that gives **the highest accuracy (coefficient of determination) during the cross-validation**

In [ ]:
best_parameters = gb_random.best_params_
print(best_parameters)  

Print the accuracy we get for the 
<span style='color:green'> <span style="font-size:15px;"> **best combination**:</span> </span>

In [ ]:
highest_accuracy = gb_random.best_score_
highest_accuracy = round(highest_accuracy,3)
highest_accuracy = highest_accuracy*100
print('Coefficient of Determination for the Gradient Boosting =', highest_accuracy)

<span style='color:green'> <span style="font-size:15px;"> *Load the results to Pandas DataFrame*:</span> </span>

In [ ]:
# Load the results to Pandas DataFrame 
results = pd.DataFrame(gb_random.cv_results_) 
results.head(20)

<span style='color:green'> <span style="font-size:15px;"> *Define the columns of interest*:</span> </span>

In [ ]:
results_of_interest = results[['param_n_estimators','param_max_depth','param_learning_rate','params','mean_test_score']]
results_of_interest.head(20)

======================================================================================================================

<span style='color:gray'> <span style="font-size:20px;">**Evaluation of the Tuned Models and Visualization of Results**</span> </span>

We consider the Test Dataset or, also called, the Hold-Out Dataset (20% of the Original Dataset) and we perform the prediction on this Dataset which is the "unseen" Dataset.

**rf_final_model**

**gb_final_model**

In [ ]:
# Create the tuned Random Forest 
rf_final_model=RandomForestRegressor(n_estimators=150, max_depth=10, random_state=42, criterion = 'squared_error')
# Create the tuned Gradient Boosting 
gb_final_model=GradientBoostingRegressor(n_estimators=250, learning_rate=0.3, max_depth=5, random_state=42)

In [ ]:
# Train the tuned Random Forest 
rf_final_model.fit(X_train, y_train)

In [ ]:
# Train the tuned Gradient Boosting 
gb_final_model.fit(X_train, y_train)

There are ( X_train ), ( X_test ), ( y_train ), ( y_test )

**Plotting ( X_test ) vs. ( y_pred_rf )**

**Plotting ( X_test ) vs. ( y_pred_gb )**


In [ ]:
# Prediction on Test data (RF)
y_pred_rf = rf_final_model.predict(X_test)

In [ ]:
# Prediction on Test data (GB)
y_pred_gb = gb_final_model.predict(X_test)

<span style='color:gray'> <span style="font-size:20px;">**PLOTS**</span> </span>

================

plot (y_test, y_pred_rf)

plot (y_test, y_pred_gb)

================

<span style='color:gray'> <span style="font-size:20px;">**Performance Regression/Evaluation Metrics**</span> </span>

Beside the "Accuracy score" which, in this case, is the "Coefficient of Determination (r2)", we can assess the goodness of fit of the models with the following Regression Metrics, by considering only the Test Dataset:

<span style='color:red'> <span style="font-size:15px;"> **Random Forest Model**</span> </span>

In [ ]:
r2 = r2_score(y_test, y_pred_rf)
r2 = round(r2,4)
r2 = r2*100
print("Coefficient of Determination:", r2)

rmse = mean_squared_error(y_test, y_pred_rf, squared = False) # By default squared is True. If True returns MSE value, if False returns RMSE value
rmse = round(rmse,4)
print("Root Mean Squared Error:", rmse)

mse = mean_squared_error(y_test, y_pred_rf, squared = True)
mse = round(mse,4)
print("Mean Squared Error:", mse)

mae = mean_absolute_error(y_test, y_pred_rf)
mae = round(mae,4)
print("Mean Absolute Error:", mae)

<span style='color:red'> <span style="font-size:15px;"> **Gradient Boosting Model**</span> </span>

In [ ]:
r2 = r2_score(y_test, y_pred_gb)
r2 = round(r2,4)
r2 = r2*100
print("Coefficient of Determination:", r2)

rmse = mean_squared_error(y_test, y_pred_gb, squared = False) # By default squared is True. If True returns MSE value, if False returns RMSE value
rmse = round(rmse,4)
print("Root Mean Squared Error:", rmse)

mse = mean_squared_error(y_test, y_pred_gb, squared = True)
mse = round(mse,4)
print("Mean Squared Error:", mse)

mae = mean_absolute_error(y_test, y_pred_gb)
mae = round(mae,4)
print("Mean Absolute Error:", mae)

<span style='color:gray'> <span style="font-size:20px;">**Selection of the Best Model**</span> </span>

<span style='color:gray'> <span style="font-size:20px;">**Feature Importance**</span> </span>

<span style='color:gray'> <span style="font-size:20px;">**Save and Load the Random Forest & Gradient Boosting Models**</span> </span>

save model and load them to use other data from other wells and reservoirs

In [ ]:
import pickle  # Library for save and load scikit-learn models

<span style='color:red'> <span style="font-size:15px;"> **Random Forest Model**</span> </span>

In [ ]:
# Define file name. ".pickle" as file extension. A pickle file is a binary file. 
filename = "random_forest.pickle"

# Save Random Forest Model by means of "pickle.dump" function to store the object data to the file. 
# This function takes 2 arguments:
# Object that you want to store.
# File object you get by opening the desired file in write-binary (wb) mode.
pickle.dump(rf_final_model, open(filename, "wb"))

# Load Random Forest Model by means of the "pickle.load" function.
# The primary argument of the function is the file object you get by opening the desired file in read-binary (rb) mode.
random_forest_model_loaded = pickle.load(open(filename, "rb")) 

In [ ]:
# To print the trained and tuned random forest model 
print(random_forest_model_loaded)

<span style='color:red'> <span style="font-size:15px;"> **Gradient Boosting Model**</span> </span>

In [ ]:
# Define file name. ".pickle" as file extension. A pickle file is a binary file. 
filename = "gradient_boosting.pickle"

# Save Random Forest Model by means of "pickle.dump" function to store the object data to the file. 
# This function takes 2 arguments:
# Object that you want to store.
# File object you get by opening the desired file in write-binary (wb) mode.
pickle.dump(gb_final_model, open(filename, "wb"))

# Load Random Forest Model by means of the "pickle.load" function.
# The primary argument of the function is the file object you get by opening the desired file in read-binary (rb) mode.
gradient_boosting_model_loaded = pickle.load(open(filename, "rb")) 

<span style='color:gray'> <span style="font-size:15px;"> **Loading entire dataset on the model**</span> </span>

You can use the loaded model to compute predictions

We perform the prediction on the entire original dataset 

In [ ]:
predictors = ["GR","NPHI"] 
X = df[predictors]
y = df["PEFZ"]

In [ ]:
# New prediction on the entire dataset
y_predicted_rf = random_forest_model_loaded.predict(X)

<span style='color:red'> <span style="font-size:15px;"> **Gradient Boosting Model**</span> </span>

<span style='color:blue'> <span style="font-size:15px;"> the Random Forest</span> </span>
and
<span style='color:blue'> <span style="font-size:15px;"> Gradient Boosting models </span> </span>